<a href="https://colab.research.google.com/github/mrteacmbk/nlp/blob/master/Train_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!unzip -uq "/content/gdrive/MyDrive/ocr.zip" -d "/content/"

In [ ]:
## Load dataset from drive into colab

!cp -R /content/gdrive/MyDrive/Datasheet/Data /content/ocr 

In [4]:
! gdown https://drive.google.com/u/0/uc?id=1T0cmkhTgu3ahyMIwGZeby612RpVdDxOR

Downloading...
From: https://drive.google.com/u/0/uc?id=1T0cmkhTgu3ahyMIwGZeby612RpVdDxOR
To: /content/ocr_dataset.zip
100% 7.71G/7.71G [02:12<00:00, 58.2MB/s]


In [5]:
! unzip -qq -o ./ocr_dataset.zip

In [6]:
import os
import shutil
import glob

In [17]:
path_dirs = '/content/'
dirs = os.listdir(path_dirs)
dir_rm = {'en_00', 'en_01', 'meta', 'vi_00', 'vi_01', 'random', 'ocr_dataset.zip'}
move_data = './ocr/'
rm_dir = 'InkData_line_processed'
for dir_ptr in dirs:
    if dir_ptr == rm_dir:
        path_dir = os.path.join(path_dirs, dir_ptr)
        count = len(os.listdir(path_dir))
        for list_file in os.listdir(path_dir):
            file = list_file.split('.')
            if list_file.endswith('.png'):
                img_file = path_dir + "/" + file[0] + ".png"
                img_file_cp = move_data + str(count).zfill(2) + ".png"
                shutil.copy2(img_file, img_file_cp)
            if list_file.endswith('.txt'):
                text_file = path_dir + "/" + file[0] + ".txt"
                text_file_cp = move_data + str(count).zfill(2) + ".txt"
                shutil.copy2(text_file, text_file_cp)
                count += 1
        shutil.rmtree(os.path.join(path_dirs, dir_ptr))
    if dir_ptr == dir_rm:
        if dir_ptr != 'ocr_dataset.zip':
            shutil.rmtree(os.path.join(path_dirs, dir_ptr))
        os.remove(dir_ptr)

list_txt = glob.glob(move_data + '*.txt')
for f in list_txt:
    os.remove(f)

In [ ]:
## Save into Drive

!cp -R /content/ocr /content/gdrive/MyDrive/Datasheet/Data

In [8]:
## Load both training,test files  into colab

!cp /content/gdrive/MyDrive/Datasheet/Data /content/

In [4]:
! pip install --quiet vietocr==0.3.5

     |████████████████████████████████| 61 kB 3.1 MB/s 
     |████████████████████████████████| 280 kB 10.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 948 kB 44.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [5]:
import matplotlib.pyplot as plt
from PIL import Image

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

In [6]:
from vietocr.tool.config import Cfg
from vietocr.model.trainer import Trainer

In [7]:
config = Cfg.load_config_from_name('vgg_seq2seq')

In [12]:
#config['vocab'] = 'aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '

dataset_params = {
    'name':'hw',
    'data_root':'/content/',
    'train_annotation':'train_annotation.txt',
    'valid_annotation':'test_annotation.txt',
    'image_height':64
}

params = {
         'print_every':200,
         'valid_every':15*200,
          'iters':20000,
          'checkpoint':'./checkpoint/transformerocr_checkpoint.pth',    
          'export':'./weights/transformerocr.pth',
          'metrics': 10000,
          'batch_size': 64
         }

config['trainer'].update(params)
config['dataset'].update(dataset_params)
config['device'] = 'cuda:1'

In [13]:
config

{'aug': {'image_aug': True, 'masked_language_model': True},
 'backbone': 'vgg19_bn',
 'cnn': {'hidden': 256,
  'ks': [[2, 2], [2, 2], [2, 1], [2, 1], [1, 1]],
  'ss': [[2, 2], [2, 2], [2, 1], [2, 1], [1, 1]]},
 'dataloader': {'num_workers': 3, 'pin_memory': True},
 'dataset': {'data_root': '/content/',
  'image_height': 64,
  'image_max_width': 512,
  'image_min_width': 32,
  'name': 'hw',
  'train_annotation': 'train_annotation.txt',
  'valid_annotation': 'test_annotation.txt'},
 'device': 'cuda:1',
 'optimizer': {'max_lr': 0.001, 'pct_start': 0.1},
 'predictor': {'beamsearch': False},
 'pretrain': {'cached': '/tmp/tranformerorc.pth',
  'id_or_url': '1nTKlEog9YFK74kPyX0qLwCWi60_YHHk4',
  'md5': 'efcabaa6d3adfca8e52bda2fd7d2ee04'},
 'quiet': False,
 'seq_modeling': 'seq2seq',
 'trainer': {'batch_size': 16,
  'checkpoint': './checkpoint/transformerocr_checkpoint.pth',
  'export': './weights/transformerocr.pth',
  'iters': 20000,
  'log': './train.log',
  'metrics': 10000,
  'print_every

In [ ]:
trainer = Trainer(config, pretrained=True)

In [ ]:
trainer.config.save('config.yml')